In [19]:
import copy
import pickle
import h5py
from scipy import spatial
from keras.models import load_model
from keras.preprocessing import image
from dragonfly import maximise_function
from style_transfer import transfer_style

/home/ktran/miniconda3/envs/art/lib/python2.7/site-packages/dragonfly/utils/oper_utils.py:30: UserWarning: cannot import name direct
Could not import Fortran direct library. Dragonfly can still be used, but might be slightly slower. To get rid of this warning, install a numpy compatible Fortran compiler (e.g. gfortran) and the python-dev package and reinstall Dragonfly.
  warn('%s\n%s'%(e, fortran_err_msg))
Using TensorFlow backend.


# Initialize
Load the information and models we need

In [21]:
art_folder = '../art_images/'

# Load the classifier
classifier = load_model('../sentiment_classification/resnet50_vso.h5')

# Initialize the data generator we need to use to feed the classifier
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=True,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=True,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=False,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Define some helper functions

In [ ]:
def calc_attribute_match(encoded_feature_vector, attribute):
    results_folder = update_style(encoded_feature_vector)

    generator = datagen.flow_from_directory(results_folder,
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode=None,
                                            shuffle=False)
    probs = classifier.predict_generator(generator, len(os.listdir(img_folder)))

    # Figure out how to calculate some objective function from the resnet output
    return foo


def update_style(encoded_feature_vector):
    # Increment the counter for specifying which folder to save the images in
    global updates
    results_folder = './style_transfer_%03d' % updates
    updates += 1

    # Fetch various information that we'll be using
    global current_image

    # Find the closest image to the vector provided, then transfer its style
    global tree
    global image_names
    _, image_index = tree.query(encoded_feature_vector)
    style_image = image_names[image_index]
    current_image = transfer_style(current_image, style_image,
                                   result_folder=results_folder)

    # Remove the style image from the list of candidates (to avoid redos)
    del image_names[image_index]
    global encoded_features
    encoded_features = np.delete(encoded_features, image_index, axis=0)

    # Update some global variables
    global style_images
    style_images.append(style_image)
    tree = spatial.KDTree(encoded_features)

    return results_folder

# Spiral out

In [ ]:
# Hyperparametercs
seed_image = 'me.jpg'
attribute = 'foo'
max_capital = 2


# Load image names and initialize the list of style images
with open(art_folder + 'image_mapping.txt', 'r') as file_handle:
    image_names = file_handle.readlines()
image_names = [name.split('\n')[0] for name in image_names]
style_images = []


# Read the encoded features, then make a KDTree from them for matching
h5f = h5py.File(art_folder + 'features.h5', 'r')
encoded_features = h5f['encoded_features'][:]
h5f.close()
tree = spatial.KDTree(encoded_features)

# Figure out the domain for dragonfly to use
maxes = encoded_features.max(axis=0)
mins = encoded_features.min(axis=0)
domain = [(min_, max_) for min_, max_ in zip(mins, maxes)]


# Let's go!
current_image = copy.deepcopy(seed_image)
updates = 0
max_val, max_pt, history = maximise_function(lambda features: calc_attribute_match(features, attribute),
                                             domain, max_capital)

# Save it before we lose it!
seed_name = seed_image.split('.')[0]
with open('history_for_%s_%s.pkl' % (attribute, seed_name), 'rb') as file_handle:
    pickle.dump(history, file_handle)